Copyright 2021 National Technology & Engineering Solutions<br>
of Sandia, LLC (NTESS). Under the terms of Contract DE-NA0003525 with NTESS,<br>
the U.S. Government retains certain rights in this software.<br>
<br>
Licensed under the Apache License, Version 2.0 (the "License");<br>
you may not use this file except in compliance with the License.<br>
You may obtain a copy of the License at<br>
<br>
   http://www.apache.org/licenses/LICENSE-2.0<br>
<br>
Unless required by applicable law or agreed to in writing, software<br>
distributed under the License is distributed on an "AS IS" BASIS,<br>
WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.<br>
See the License for the specific language governing permissions and<br>
limitations under the License.

In [1]:
import logging

In [43]:
import numpy

In [44]:
import cicada.communicator
import cicada.encoder
import cicada.additive
logging.basicConfig(level=logging.INFO)

In [45]:
numTruncBits =16
testVal = 1
expectedPretrunc = testVal*2**(numTruncBits)

In [46]:
@cicada.communicator.NNGCommunicator.run(world_size=3)
def main(communicator):
    log = cicada.Logger(logging.getLogger(), communicator)    
    protocol = cicada.additive.AdditiveProtocol(communicator)
    secret2mult = protocol.share(src=0, secret=protocol.encoder.encode(numpy.array(testVal)), shape=())
    revealedsecret = protocol.encoder.decode(protocol.reveal(secret2mult))
    log.info(f"Player {communicator.rank} revealed: {revealedsecret} expected: {testVal}")
    secretMuld = protocol.untruncated_multiply(secret2mult, secret2mult)
    revealedSecretMuld = protocol.encoder.decode(protocol.reveal(secretMuld))
    log.info(f"Player {communicator.rank} revealed: {revealedSecretMuld} expected: {expectedPretrunc}")
    secretMuldTruncd = protocol.truncate(secretMuld)
    revealedSecretMuldTruncd = protocol.encoder.decode(protocol.reveal(secretMuldTruncd))
    log.info(f"Player {communicator.rank} revealed: {revealedSecretMuldTruncd} expected: {testVal}")

In this example, we secret share 1 encoded in the field (this amounts to shifting 16 bits left since that is the precision we use for our practional representation). We then multiply 1 by itself which, due to our symantic field interpretation shifts the correct answer by 16 additional bits. This is the reason we expect 65536 (2^16) to be revealed prior to applying tuncation. 

Once we apply truncation we reveal the result and, as expected, reconstruct 1 after decoding, as we would wish for multiplying 1 by itself. 

In [47]:
main()

INFO:cicada.communicator.nng:Player 0 rendezvous with tcp://127.0.0.1:63894 from tcp://127.0.0.1:63894.
INFO:cicada.communicator.nng:Player 1 rendezvous with tcp://127.0.0.1:63894 from tcp://127.0.0.1:63895.
INFO:cicada.communicator.nng:Player 2 rendezvous with tcp://127.0.0.1:63894 from tcp://127.0.0.1:63896.
INFO:cicada.communicator.nng:Comm 'world' player 0 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 1 communicator ready.
INFO:cicada.communicator.nng:Comm 'world' player 2 communicator ready.
INFO:root:Player 0 revealed: 1.0 expected: 1
INFO:root:Player 1 revealed: 1.0 expected: 1
INFO:root:Player 2 revealed: 1.0 expected: 1
INFO:root:Player 0 revealed: 65536.0 expected: 65536
INFO:root:Player 1 revealed: 65536.0 expected: 65536
INFO:root:Player 2 revealed: 65536.0 expected: 65536
INFO:root:Player 0 revealed: 1.0 expected: 1
INFO:root:Player 1 revealed: 1.0 expected: 1
INFO:root:Player 2 revealed: 1.0 expected: 1
INFO:cicada.communicator.nng:Comm 'world' play

[None, None, None]